In [1]:
import json
import os

from dotenv import load_dotenv, find_dotenv
from langchain_core.prompt_values import PromptValue
from langchain_openai import ChatOpenAI
from tqdm import tqdm
from tqdm.asyncio import tqdm_asyncio

from agent.utils.loader import load_prompt, load_processed_data
from agent.utils.tools.google_search import GoogleSearchTool

_ = load_dotenv(find_dotenv())

os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_PROJECT"] = "self-correct"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"

In [2]:
MAX_ITERATION = 7

dataset_name = 'hotpot_qa'
mode = "critic"
model = "gpt-4o-mini-2024-07-18"
num_samples = 1000
if dataset_name in ['hotpot_qa', 'ambig_qa', 'trivia_qa']:
	google_search = GoogleSearchTool()
	temperature = 0
	n = 1
elif dataset_name in ['gsm8k', 'svamp', 'tabmwp']:
	temperature = 0.5
	n = 1
else:
	temperature = 0.9
	n = 25
	model = "gpt-4o-mini"
top_p = 0.95
seed = 42
batch_size = 50
processed_data_path = f"../../../output/inference/{model}/{dataset_name}/cot/num_samples_{num_samples}_top_p_{top_p}_temperature_0_seed_{seed}.jsonl"
save_results_path = f"../../../output/inference/{model}/{dataset_name}/{mode}/num_samples_{num_samples}_top_p_{top_p}_temperature_{temperature}_seed_{seed}.jsonl"
prompt = load_prompt(dataset_name=dataset_name, mode=mode)
dataset = load_processed_data(dataset_name=dataset_name, file_path=processed_data_path).map(
	lambda x: {"generation": x["generation"][3:]})
if num_samples > 0:
	dataset = dataset.select(range(num_samples))
llm = ChatOpenAI(model_name="gpt-4o-mini-2024-07-18",
                 top_p=top_p,
                 n=n,
                 temperature=temperature,
                 openai_api_base="https://api.chsdw.top/v1",
                 seed=seed,
                 max_retries=3)

prompt.pretty_print()

print(dataset[2])

================================ System Message ================================

Your response should follow the previous format and style.

================================== AI Message ==================================

Question: Serianna is a band of what genre that combines elements of heavy metal and hardcore punk?
Proposed Answer: Let's think step by step. Serianna is a band of metalcore genre. Metalcore is a subgenre of heavy metal and hardcore punk. So Serianna is a band of heavy metal and hardcore punk. FINAL ANSWER: heavy metal and hardcore punk.

What's the problem with the above answer?

1. Plausibility:

The question asks for the genre that combines elements of heavy metal and hardcore punk, and the answer is "heavy metal and hardcore punk", simply repeat the question. So it's not plausible.

2. Truthfulness:

Let's search the question in google:
> Search Query: Serianna is a band of what genre that combines elements of heavy metal and hardcore punk? site: wikipedia.org


In [3]:
import logging

# 配置 logging
logging.basicConfig(
	level=logging.WARNING,
	format="%(asctime)s - %(levelname)s - %(message)s",  # 日志格式
    datefmt="%Y-%m-%d %H:%M:%S",  # 时间格式
)

async def critic_iter(item: dict, previous_cot: str):
	# load prompt
	prompt_critic: PromptValue = prompt.invoke(input={**item, "previous_cot": previous_cot})
	logging.debug(prompt_critic.to_string())

	# verify: plausible & truthful
	context = "What's the problem with the above answer?\n\n1. Plausibility:\n\n"
	prompt_critic.messages[-1].content += context
	logging.debug(context)


	exist_query = []
	exist_evidence = set()
	revised_cot = ""
	for idx in range(MAX_ITERATION):  # max interaction with tool
		# print("\n\n" + "=" * 30, "Round", idx, "=" * 30)
		# get LLM res
		try:
			res = await llm.ainvoke(input=prompt_critic, stop=["> Evidence:", "---"])
			res = res.content
		except Exception as e:
			logging.error("Error when invoking LLM.")
			raise e

		# case1: search
		if "> Search Query:" in res:
			logging.debug("CASE 1:")
			try:
				_, search_query = res.split("> Search Query:")[:2]
				search_query = search_query.split("\n")[0].strip()
			except:
				# print("Search Query Error:", res)
				exit()

			prompt_critic.messages[-1].content += res
			# print(res, end="")

			# if args.use_tool:
			# use Tool: search a new evidence
			# exist_query.append(search_query)
			# for k in range(exist_query.count(search_query), 8):
			search_res: list = await google_search.arun(search_query)
			search_res = search_res[0]
			# 	if search_res['link'] not in exist_evidence:
			# 		exist_evidence.add(search_res['page'])
			# 		break

			try:
				context = f"""> Evidence: [{search_res}]\n\n"""
			except:
				context = f"""> Error when trying to search evidence.\n\n"""
			if idx == MAX_ITERATION - 2:
				context += f"Let's give the most possible answer.\n\nQuestion: {item['question']}\nHere's "
			# else:
			#     # w/o Tool: use LLMs generated evidence
			#     context = """> Evidence: """
			# print(context, end="")
			prompt_critic.messages[-1].content += context


		# case2: most possible answer
		elif "most possible answer:" in res:
			# print("CASE 2:")
			# print(res)
			_, revised_cot = res.split("most possible answer:")
			revised_cot = revised_cot.strip()
			prompt_critic.messages[-1].content += revised_cot
			# print(prompt_critic.to_string())
			break
		# case3: other output
		else:
			# print("CASE 3:")
			if not res:
				print("NOT A RESPONSE.")
				break
			context = res
			context += f"\nLet's give the most possible answer.\n\nQuestion: {item['question']}\nHere's "
			# print(context, end="")
			prompt_critic.messages[-1].content += context

	return revised_cot


def is_null_answer(text):
	if not text:
		return True
	text = text.strip().lower()
	if text in ["none", "", "no answer", "never", "null", "both", "neither"]:
		return True
	if text.startswith("none"):
		return True
	return False


async def critic(item: dict):
	# print(f"Question: {item['question']}")
	# print(f"Gold answer: {item['answer']}")
	# iterative correction
	previous_corrected = True
	for itr in range(1, 4):
		# initialization
		if itr == 1:
			# extract prediction
			init_cot = item['generation']
			# 返回答案部分
			init_pred = item['prediction']

			# cot and pred
			item['cot'] = [init_cot]
			item['pred'] = [init_pred]

		# choose the latest answer that is not "None" to critic
		base_idx = itr - 1
		while base_idx > 0 and is_null_answer(item['pred'][base_idx]):
			base_idx -= 1
		previous_cot = item['cot'][base_idx]
		previous_pred = item['pred'][base_idx]

		# one iteration
		revised_cot = await critic_iter(item, previous_cot)
		revised_pred = revised_cot.split("FINAL ANSWER:")[-1].strip()

		# is corrected
		corrected = True
		if revised_pred and (revised_pred == previous_pred):
			corrected = False

		item['cot'].append(revised_cot)
		item['pred'].append(revised_pred)

		# if no correction for twice, break
		if not corrected and not previous_corrected:
			print("Stop.")
			break
		previous_corrected = corrected

	return item

In [6]:
results = []
async def inference() -> None:
	if os.path.exists(save_results_path):
		with open(save_results_path, 'r') as file:
			for line in file:
				results.append(json.loads(line))
	else:
		folder_path = os.path.dirname(save_results_path)
		os.makedirs(folder_path, exist_ok=True)

	for idx in tqdm(range(len(results), dataset.num_rows, batch_size)):
		batch = dataset.select(range(idx, min(idx + batch_size, dataset.num_rows)))
		results.extend(await tqdm_asyncio.gather(*(critic(item) for item in batch)))
		with open(save_results_path, 'qa') as file:
			for result in results:
				file.write(json.dumps(result) + "\n")

In [7]:
await inference()

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  2%|▏         | 1/50 [00:12<09:57, 12.19s/it]

Stop.




  4%|▍         | 2/50 [00:12<04:07,  5.16s/it]

  6%|▌         | 3/50 [00:12<02:13,  2.85s/it]

Stop.
Stop.




  8%|▊         | 4/50 [00:14<01:46,  2.31s/it]

...



 66%|██████▌   | 33/50 [03:34<00:45,  2.70s/it]

 68%|██████▊   | 34/50 [03:44<01:18,  4.88s/it]

 70%|███████   | 35/50 [03:46<01:01,  4.09s/it]

 72%|███████▏  | 36/50 [03:46<00:41,  2.98s/it]2024-12-24 23:07:58 - WARNING - Client error occurred: Cannot connect to host www.googleapis.com:443 ssl:default [None], retrying 1...


 74%|███████▍  | 37/50 [03:52<00:51,  3.92s/it]

 76%|███████▌  | 38/50 [03:56<00:47,  3.98s/it]

 78%|███████▊  | 39/50 [03:58<00:36,  3.30s/it]

Stop.




 80%|████████  | 40/50 [04:01<00:30,  3.03s/it]

 82%|████████▏ | 41/50 [04:02<00:22,  2.51s/it]

 84%|████████▍ | 42/50 [04:03<00:16,  2.00s/it]

 86%|████████▌ | 43/50 [04:05<00:15,  2.16s/it]

 88%|████████▊ | 44/50 [04:07<00:12,  2.06s/it]

 90%|█████████ | 45/50 [04:09<00:09,  1.96s/it]

 92%|█████████▏| 46/50 [04:09<00:06,  1.52s/it]

 94%|█████████▍| 47/50 [04:10<00:03,  1.21s/it]

 96%|█████████▌| 48/50 [04:11<00:02,  1.35s/it]

Stop.




 98%|█████████▊| 49/50 [04:23<00:04,  4.48s/it]

100%|██████████| 3/3 [10:59<00:00, 219.87s/it]
